In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


# Data

In [2]:
df_train = pd.read_csv('../input/titanic/train.csv')
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df_test = pd.read_csv('../input/titanic/test.csv')
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
df_train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Fare', 'Embarked'], axis = 1, inplace=True)
df_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,male,22.0,1,0
1,1,1,female,38.0,1,0
2,1,3,female,26.0,0,0
3,1,1,female,35.0,1,0
4,0,3,male,35.0,0,0


In [5]:
submission = pd.DataFrame()
submission['PassengerId'] = df_test['PassengerId']
submission.head()

,PassengerId
0,892
1,893
2,894
3,895
4,896


In [6]:
df_test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Fare', 'Embarked'], axis = 1, inplace=True)
df_test.head()

,Pclass,Sex,Age,SibSp,Parch
0,3,male,34.5,0,0
1,3,female,47.0,1,0
2,2,male,62.0,0,0
3,3,male,27.0,0,0
4,3,female,22.0,1,1


In [7]:
df_train.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
dtype: int64

In [8]:
df_train.groupby(df_train['Age'].isnull()).Survived.mean()

Age
False    0.406162
True     0.293785
Name: Survived, dtype: float64

In [9]:
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].mean())
df_train['Age'].isnull().sum()

0

In [10]:
df_test.isnull().sum()

Pclass     0
Sex        0
Age       86
SibSp      0
Parch      0
dtype: int64

In [11]:
df_test['Age'] = df_test['Age'].fillna(df_train['Age'].mean())
df_test['Age'].isnull().sum()

0

In [12]:
df_train.loc[df_train['Sex'] == 'male', 'Sex'] = 0
df_train.loc[df_train['Sex'] == 'female', 'Sex'] = 1
df_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,0,22.0,1,0
1,1,1,1,38.0,1,0
2,1,3,1,26.0,0,0
3,1,1,1,35.0,1,0
4,0,3,0,35.0,0,0


In [13]:
df_test.loc[df_test['Sex'] == 'male', 'Sex'] = 0
df_test.loc[df_test['Sex'] == 'female', 'Sex'] = 1
df_test.head()

,Pclass,Sex,Age,SibSp,Parch
0,3,0,34.5,0,0
1,3,1,47.0,1,0
2,2,0,62.0,0,0
3,3,0,27.0,0,0
4,3,1,22.0,1,1


In [14]:
train_y = df_train['Survived']
train_y = np.array(train_y)

In [15]:
df_train.drop(['Survived'], axis = 1, inplace = True)
df_train.head()

,Pclass,Sex,Age,SibSp,Parch
0,3,0,22.0,1,0
1,1,1,38.0,1,0
2,3,1,26.0,0,0
3,1,1,35.0,1,0
4,3,0,35.0,0,0


In [16]:
train_x = np.array(df_train)
print(np.shape(train_x))

(891, 5)


In [17]:
test = np.array(df_test)
print(np.shape(test))

(418, 5)


# Training

In [18]:
from keras import models, layers
from keras.callbacks import EarlyStopping

x = layers.Input((5,))
h = layers.Dense(4)(x)
o = layers.Dense(1, activation = 'relu')(h)

model = models.Model(x, o)
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy')
early_stopping = EarlyStopping(monitor='loss', mode = 'min', restore_best_weights = True, patience = 3)
model.fit(train_x, train_y, epochs = 100, batch_size = 10, callbacks = [early_stopping])

Using TensorFlow backend.


Epoch 1/100
891/891 [==============================] - 0s 481us/step - loss: 5.9064
Epoch 2/100
891/891 [==============================] - 0s 124us/step - loss: 5.9060
Epoch 3/100
891/891 [==============================] - 0s 127us/step - loss: 5.9057
Epoch 4/100
891/891 [==============================] - 0s 128us/step - loss: 5.9055
Epoch 5/100
891/891 [==============================] - 0s 126us/step - loss: 5.9053
Epoch 6/100
891/891 [==============================] - 0s 142us/step - loss: 5.9051
Epoch 7/100
891/891 [==============================] - 0s 128us/step - loss: 5.9050
Epoch 8/100
891/891 [==============================] - 0s 133us/step - loss: 5.8908
Epoch 9/100
891/891 [==============================] - 0s 128us/step - loss: 5.8766
Epoch 10/100
891/891 [==============================] - 0s 124us/step - loss: 5.8611
Epoch 11/100
891/891 [==============================] - 0s 126us/step - loss: 5.8458
Epoch 12/100
891/891 [==============================] - 0s 126us/step - lo

# Predict

In [19]:
result = model.predict(test)

In [20]:
submission['Survived'] = result
submission.head()

,PassengerId,Survived
0,892,0.108059
1,893,0.564890
2,894,0.126651
3,895,0.123437
4,896,0.641339


In [21]:
submission['Survived'] = round(submission['Survived']).astype(int)
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [22]:
submission.to_csv('titanic_ann.csv', index = False)